In [21]:
from PIL import Image 
import random
from numpy import array 
import numpy as np

img = Image.open('../Lena.png')

ar = array(img)

In [22]:
ar

array([[160, 160, 160, ..., 169, 154, 129],
       [161, 160, 159, ..., 168, 154, 129],
       [160, 161, 160, ..., 168, 154, 129],
       ...,
       [ 50,  49,  55, ..., 106, 103, 104],
       [ 49,  50,  58, ..., 107, 109, 111],
       [ 50,  49,  59, ..., 107, 108, 112]], dtype=uint8)

In [23]:
ar2 = np.transpose(ar)

In [24]:
print(ar2)

[[160 161 160 ...  50  49  50]
 [160 160 161 ...  49  50  49]
 [160 159 160 ...  55  58  59]
 ...
 [169 168 168 ... 106 107 107]
 [154 154 154 ... 103 109 108]
 [129 129 129 ... 104 111 112]]


In [25]:
def shuffle_arr(arr):
    row = len(arr)
    col = len(arr[0])
    for i in range(0,row):
        for j in range(0,col):
            arr[i][j] = ((row*row)+i+1)%256
    for i in range(0,int(row/4)):
        for j in range(int(col/4)):
            arr[i][j] = (row*row + 1)%256 - arr[i][j]
    for i in range(0, int(row/4)):
        for j in range(3*int(col/3),col):
            arr[i][j] = ((row*row)+1)%256-arr[i][j]
    for i in range(3*(int(row/4)),row):
        for i in range(0,int(col/4)):
            arr[i][j] = (row*row+1)%256 - arr[i][j]
    for i in range(int(row/4),3*int(row/4)):
        for j in range(int(col/4),3*int(col/4)):
            arr[i][j] = (row*row+1)%256 -arr[i][j]
    
    temp = np.array(arr)

    for i in range(0,row):
        for j in range(0,col):
            rowindex = int((arr[i][j]-1)%row)
            colindex = int((arr[i][j]-1)/col)
            temp[i][j] = arr[rowindex][colindex]

    return temp

In [26]:
ar3 = shuffle_arr(ar2)

In [27]:
ar3

array([[  0,   0,   0, ...,   0,   0,   0],
       [255, 255, 255, ..., 255, 255, 255],
       [254, 254, 254, ..., 254, 254, 254],
       ...,
       [254, 254, 254, ..., 254, 254, 254],
       [255, 255, 255, ..., 255, 255, 255],
       [  0,   0,   0, ...,   0,   0,   0]], dtype=uint8)

In [28]:
shuffled_img3 = Image.fromarray(ar3)

In [29]:
shuffled_img3.save('shuffeled_image3.png')

In [30]:
def key_gen(r,c):
    R = 3.99
    random_float = random.uniform(0,1)
    x = 0.4+(random_float/5)
    key = np.zeros((r,c))
    for i in range(0,r):
        for j in range(0,c):
            x = x*R*(1-x)
            key[i][j] = x*255
    
    return key

In [31]:
r = len(ar3)
c = len(ar3[0])
best_key = key_gen(r,c)
print(best_key)

[[247.70645563  28.26888438 100.28881731 ...  59.69119166 182.41678389
  207.17326831]
 [155.03763554 242.4970009   47.44094016 ...  11.10759831  42.38880284
  141.01652208]
 [251.50383916  13.75842554  51.93421926 ... 182.03902776 207.82023671
  153.41799689]
 ...
 [ 22.25798709  81.05753408 220.6133175  ...  47.27563907 153.6588417
  243.65545236]
 [ 43.25098798 143.30130335 250.45572377 ...  48.39568976 156.45115619
  241.2476134 ]
 [ 51.91272346 164.96402496 232.40102105 ...  60.33718771 183.78106869
  204.79952281]]


In [32]:
def encrypt(ar3,key):
    r = len(ar3)
    c = len(ar3[0])
    for i in range(0,r):
        for j in range(0,c):
            ar3[i][j] = int(ar3[i][j]) ^ int(key[i][j])
            
    cipher_img = np.array(ar3)
    for itr in range(0,int(r/16)):
        for i in range(0,r):
            for j in range(0,c):
                xmap = int((i+j)%r)
                ymap = int((i+2*j)%c)
                cipher_img[i][j] = cipher_img[xmap][ymap]
                
    
    
    return ar3

In [33]:
encrypted_img_arr = encrypt(ar3,best_key)
print(encrypted_img_arr)

[[247  28 100 ...  59 182 207]
 [100  13 208 ... 244 213 114]
 [  5 243 205 ...  72  49 103]
 ...
 [232 175  34 ... 209 103  13]
 [212 112   5 ... 207  99  14]
 [ 51 164 232 ...  60 183 204]]


In [34]:
encrypted_img = Image.fromarray(encrypted_img_arr)

In [35]:
encrypted_img.save('encrypted_img.png')

In [36]:
def correlation_coefficient(img1, img2):
    # Ensure that the images have the same shape
    assert img1.shape == img2.shape, "The images have different shapes"
    
    # Calculate the mean of the images
    mean1 = np.mean(img1)
    mean2 = np.mean(img2)
    
    # Calculate the standard deviation of the images
    std1 = np.std(img1)
    std2 = np.std(img2)
    
    # Subtract the mean from each pixel and divide by the standard deviation
    normalized_img1 = (img1 - mean1) / std1
    normalized_img2 = (img2 - mean2) / std2
    
    # Calculate the sum of the product of the corresponding pixels
    sum_of_products = np.sum(normalized_img1 * normalized_img2)
    
    # Calculate the correlation coefficient
    return sum_of_products / (img1.size - 1)

In [37]:
correlation_coefficient(ar,encrypted_img_arr)


-0.0012242905973564185

In [38]:

def NPCR(img1, img2):
    """
    Calculate the NPCR value
    Bigger the better
    [0-100]
    Parameters:
        img1 = Encrypted image 1
        img2 = Encrypted image 2 with one pixel changed
    Returns:
        NPCR value
     """
    height, width = img1.shape
    sum = 0
    for i in range(height):
        for j in range(width):
            if img1[i, j] != img2[i, j]:
                sum += 1
    res = ((sum)/(width*height))*100
    return res


def UACI(img1, img2):
    """
    Calculate the UACI value
    Bigger the Better
    Parameters:
        img1 = Encrypted image 1
        img2 = Encrypted image 2 with one pixel changed
    Returns:
        UACI value
    """
    height, width = img1.shape
    difference = 0
    for y in range(height):
        for x in range(width):
            difference += (abs(int(img1[x, y])-int(img2[x, y])))
    difference = ((difference*100)/(width*height*255))
    return difference


def NPCR_UACI_worker(img1, img2):
    """
    Calculate the NPCR & UACI worker
    Parameters:
        img1 = Encrypted image 1
    Returns:
        NPCR value
    """
    return NPCR(img1, img2), UACI(img1, img2)


In [39]:
NPCR_UACI_worker(ar,encrypted_img_arr)

(99.60823059082031, 33.70198268516391)